In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from PINNs_Chron import PINN3D, PINN3DSolver

# Set data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE)
tf.get_logger().setLevel('ERROR')

In [ ]:
# tf.config.list_physical_devices('GPU')

In [ ]:
%matplotlib inline

SIZE = 12
BIGGER_SIZE = 16

plt.rc('font', family='Arial', size=SIZE) # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize

In [ ]:
uplift = lambda t : np.where(t < 40, .6, .05)
tf_uplift = lambda t: tf.where(t < 40, .6, .05)

In [ ]:
topo_ar = np.loadtxt('dabie_utm_km.xyz', dtype=DTYPE)
topo_x, topo_x_pos = np.unique(topo_ar[:, 0], return_inverse=True)
topo_y, topo_y_pos = np.unique(topo_ar[:, 1], return_inverse=True)
topo_pivot = np.zeros((len(topo_x), len(topo_y)), dtype=DTYPE)
topo_pivot[topo_x_pos, topo_y_pos] = topo_ar[:, 2]
topo_x_min, topo_x_max = topo_x.min(), topo_x.max()
topo_y_min, topo_y_max = topo_y.min(), topo_y.max()

In [ ]:
t_end = 150.
h0 = 30.
xl = topo_x.max() - topo_x.min()
yl = topo_y.max() - topo_y.min()
Tsea = 15.
Tbot = 600.
kappa = 25.
air_lapse = 5.

In [ ]:
import tensorflow_probability as tfp
def tf_h_fn(t, x, y):
    amp = tf.where(t < 70., 4., 1. + (4. - 1.) * (t_end - t)/ (t_end - 70.))
    xy = tf.cast(tf.concat([x, y], axis=1), dtype=DTYPE)
    h_interp = tfp.math.batch_interp_regular_nd_grid(xy, [topo_x_min, topo_y_min,], [topo_x_max, topo_y_max], topo_pivot, axis=-2)
    return h0 + amp * tf.reshape(h_interp, tf.shape(x))

In [ ]:
from scipy.interpolate import LinearNDInterpolator
def scipy_h_fn(t, x, y):
    amp = np.where(t < 70., 4., 1. + (4. - 1.) * (t_end - t)/ (t_end - 70.))
    interp = LinearNDInterpolator(list(zip(topo_ar[:, 0], topo_ar[:, 1])), topo_ar[:, 2])
    return h0 + amp * interp(x, y)

In [ ]:
# Define the initial condition
T_grad = lambda t, x, y: (Tbot - T_surf(tf_h_fn(t, x, y))) / tf_h_fn(t, x, y)
T_grad0 = lambda x, y: T_grad(0, x, y)

def T_init(x, y, z):
    return Tbot - T_grad0(x, y) * z

# Define boundary condition
def T_surf(surf_z, sealevel_z=h0, lapse=air_lapse):
    elevation = surf_z - sealevel_z
    return tf.ones(tf.shape(surf_z), dtype=DTYPE) * Tsea - elevation * lapse

def T_bot(z):
    return tf.constant(Tbot, shape=z.shape, dtype=DTYPE)

In [ ]:
# Set number of data points
N_0 = 2000
N_b_b = 2000
N_b_s = 10000
N_b = N_b_b + N_b_s
N_r = 100000

# Set boundary
tmin = 0.
tmax = t_end
zmin = 0.
zmax = h0 + topo_ar[:, 2].max() * 4.
xmin = 0.
xmax = xl
ymin = 0
ymax = yl

# Lower bounds
lb = tf.constant([tmin, xmin, ymin, zmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([tmax, xmax, ymax, zmax], dtype=DTYPE)

# Set random seed
# tf.random.set_seed(128)

# Draw uniform sample points for initial boundary data
t_0 = tf.ones((N_0, 1), dtype=DTYPE) * lb[0]
x_0 = tf.random.uniform((N_0, 1), lb[1], ub[1], dtype=DTYPE)
y_0 = tf.random.uniform((N_0, 1), lb[2], ub[2], dtype=DTYPE)
z_0 = tf.random.uniform((N_0, 1), lb[3], tf_h_fn(0., x_0, y_0), dtype=DTYPE)
X_0 = tf.concat([t_0, x_0, y_0, z_0], axis=1)
# Evaluate intitial condition at z_0
T_0 = T_init(x_0, y_0, z_0)

# Boundary data
mid_t = lb[0]+(ub[0]-lb[0])*.27 #to separate t space into two and sample each
# t_b = tf.random.uniform((N_b_b, 1), lb[0], ub[0], dtype=DTYPE)
t_b0 = tf.random.uniform((N_b_b//2, 1), lb[0], mid_t, dtype=DTYPE)
t_b1 = tf.random.uniform((N_b_b//2, 1), mid_t, ub[0], dtype=DTYPE)
t_b = tf.concat([t_b0, t_b1], axis=0)

x_b = tf.random.uniform((N_b_b, 1), lb[1], ub[1], dtype=DTYPE)
y_b = tf.random.uniform((N_b_b, 1), lb[2], ub[2], dtype=DTYPE)
z_b = lb[3] * tf.ones((N_b_b, 1), dtype=DTYPE)

# t_s = tf.random.uniform((N_b_s, 1), lb[0], ub[0], dtype=DTYPE)
t_s0 = tf.random.uniform((N_b_s//2, 1), lb[0], mid_t, dtype=DTYPE)
t_s1 = tf.random.uniform((N_b_s//2, 1), mid_t, ub[0], dtype=DTYPE)
t_s = tf.concat([t_s0, t_s1], axis=0)

x_s = tf.random.uniform((N_b_s, 1), lb[1], ub[1], dtype=DTYPE)
y_s = tf.random.uniform((N_b_s, 1), lb[2], ub[2], dtype=DTYPE)
z_s = tf.constant(tf_h_fn(t_s, x_s, y_s), dtype=DTYPE)

t_bs = tf.concat([t_b, t_s], axis=0)
x_bs = tf.concat([x_b, x_s], axis=0)
y_bs = tf.concat([y_b, y_s], axis=0)
z_bs = tf.concat([z_b, z_s], axis=0)
X_bs = tf.concat([t_bs, x_bs, y_bs, z_bs], axis=1)

# Evaluate boundary condition
T_b = T_bot(z_b)
T_s = T_surf(z_s)
T_bs = tf.concat([T_b, T_s], axis=0)

# Draw uniformly sampled collocation points
# t_r = tf.random.uniform((N_r, 1), lb[0], ub[0], dtype=DTYPE)
t_r0 = tf.random.uniform((N_r//2, 1), lb[0], mid_t, dtype=DTYPE)
t_r1 = tf.random.uniform((N_r//2, 1), mid_t, ub[0], dtype=DTYPE)
t_r = tf.concat([t_r0, t_r1], axis=0)

x_r = tf.random.uniform((N_r, 1), lb[1], ub[1], dtype=DTYPE)
y_r = tf.random.uniform((N_r, 1), lb[2], ub[2], dtype=DTYPE)
z_r = tf.random.uniform((N_r, 1), lb[3], tf_h_fn(t_r, x_r, y_r), dtype=DTYPE)
X_r = tf.concat([t_r, x_r, y_r, z_r], axis=1)

# Collect boundary and inital data in lists
X_data = tf.concat([X_0, X_bs], axis=0)
T_data = tf.concat([T_0, T_bs], axis=0)

In [ ]:
import matplotlib.colors as colors
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = truncate_colormap(plt.get_cmap('plasma'), 0, .8)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 3), ncols=3,
                        width_ratios=(1, 1, 1.2),
                        subplot_kw={'projection': '3d'})

axs[0].scatter(X_0.numpy()[:, 1], X_0.numpy()[:, 2], X_0.numpy()[:, 3],
                    c=T_0.numpy(), marker='x', s=15, cmap=cmap,
                    vmin=0, vmax=600)
im = axs[1].scatter(X_bs.numpy()[:, 1], X_bs.numpy()[:, 2], X_bs.numpy()[:, 3],
                    c=T_bs.numpy(), marker='+', s=25, cmap=cmap,
                    vmin=0, vmax=600)
axs[2].scatter(x_r, y_r, z_r, c='dimgray', marker='o', edgecolor='w')
x_mesh, y_mesh = np.meshgrid(np.linspace(0, 160, 20), np.linspace(0, 200, 25))
z_mesh = scipy_h_fn(0., x_mesh, y_mesh)
for ax in axs:
    for c, w in zip(['w', 'slategray'], [1, .1]):
        ax.plot_surface(x_mesh, y_mesh, z_mesh, color='None',
                       edgecolor=c, linewidth=w, antialiased=False)
        ax.plot_surface(x_mesh, y_mesh, np.zeros_like(z_mesh), color='None',
                       edgecolor=c, linewidth=w, antialiased=False)

for ax, s in zip(axs, 'abc'):
    # ax.set_xticklabels([])
    # ax.set_yticklabels([])
    # ax.set_zticklabels([])
    ax.tick_params(axis='both', pad=-2)
    ax.set_aspect('equalxy')
    ax.grid(False)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.set_xlim(0, 160)
    ax.set_ylim(0, 200)
    ax.set_zlim(0, 30)
    ax.set_xlabel('X (km)', labelpad=-2)
    ax.set_ylabel('Y (km)', labelpad=-2)
    ax.text(0, 0, 38, s, fontsize=14)
    
axs[-1].set_zlabel('Z (km)', labelpad=-2)
    
cb = fig.colorbar(im, ax=axs[-1], shrink=0.5, anchor=(1, .5))
cb.ax.invert_yaxis()
cb.set_label('Temperature (°C)')

fig.tight_layout()
# fig.savefig('GChron1/3d_icbc.pdf')
# fig.savefig('GChron1/3d_icbc.png', dpi=300)

In [ ]:
from time import time
# Initialize model
model = PINN3D(lb, ub, num_hidden_layers=16, num_neurons_per_layer=20)
model.Tgrad = T_grad
model.build(input_shape=(None, 4))

# Initilize PINN solver
solver = PINN3DSolver(model, X_r)
solver.fun_u = tf_uplift
solver.savepath = 'saved_model3d'

# Start timer
t0 = time()
# Solving with Adam
# lr = 1e-3
# lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([10000, 50000],[1e-2, 1e-3, 1e-4])
lr = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, decay_steps=10000, decay_rate=0.9)
optim = tf.keras.optimizers.Adam(learning_rate=lr)
solver.solve_with_Adam(optim, X_data, T_data, N=400000, echofreq=1000, savefreq=10000)

# Print computation time
print('\nComputation time: {} seconds'.format(time()-t0))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(solver.loss_hist, label='$\phi_{total}$')
ax.plot(solver.heat_hist, label='$\phi_{heat}$')
ax.plot(solver.bound_hist, label='$\phi_{ibc}$')
ax.set_yscale('log')
ax.set_yticks(np.logspace(0, 5, 6))
ax.legend(loc='upper right')
ax.set_xlabel('Iteration')
ax.set_ylabel('Loss')

fig.tight_layout()